In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso, LassoCV, Ridge, RidgeCV
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import numpy as np

### Carregar e visualizar os dados - Regressão

In [4]:
df = pd.read_csv('datasets\colesterol.csv')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Id                  1000 non-null   int64  
 1   Grupo Sanguíneo     1000 non-null   object 
 2   Fumante             1000 non-null   object 
 3   Nível de Atividade  1000 non-null   object 
 4   Idade               1000 non-null   int64  
 5   Peso                1000 non-null   float64
 6   Altura              1000 non-null   int64  
 7   Colesterol          1000 non-null   float64
dtypes: float64(2), int64(3), object(3)
memory usage: 62.6+ KB


In [6]:
df.drop('Id', axis=1, inplace=True)
df = pd.get_dummies(df, columns=['Grupo Sanguíneo', 'Fumante', 'Nível de Atividade'])

In [7]:
df

,Idade,Peso,Altura,Colesterol,Grupo Sanguíneo_A,Grupo Sanguíneo_AB,Grupo Sanguíneo_B,Grupo Sanguíneo_O,Fumante_Não,Fumante_Sim,Nível de Atividade_Alto,Nível de Atividade_Baixo,Nível de Atividade_Moderado
0,33,85.1,186,199.63,0,0,1,0,0,1,0,1,0
1,68,105.0,184,236.98,1,0,0,0,1,0,0,0,1
2,25,64.8,180,161.79,0,0,0,1,1,0,1,0,0
3,43,120.2,167,336.24,1,0,0,0,1,0,1,0,0
4,79,88.5,175,226.23,0,1,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,31,68.1,166,206.81,0,1,0,0,0,1,0,0,1
996,51,47.7,170,128.03,0,0,0,1,1,0,1,0,0
997,39,85.5,176,211.14,0,1,0,0,1,0,0,1,0
998,61,91.2,161,284.53,0,1,0,0,0,1,0,1,0


## Treinar modelo de regressão com Lasso (L1)

In [8]:
X = df.drop('Colesterol', axis=1)
y = df['Colesterol']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=51)

In [13]:
model_lasso = Lasso(alpha=0.1)
model_lasso.fit(X_train, y_train)

Lasso(alpha=0.1)

In [14]:
def importancia_features(modelo):
    importance = np.abs(modelo.coef_)
    print("Importância das features:")
    for i, feature in enumerate(modelo.feature_names_in_):
        print(f'{feature}: {importance[i]}')   

In [15]:
# importância das features - Lasso
importancia_features(model_lasso)

Importância das features:
Idade: 0.02398570771151734
Peso: 2.4765870268702703
Altura: 2.201760869534116
Grupo Sanguíneo_A: 0.12855801865958197
Grupo Sanguíneo_AB: 0.9304059600403655
Grupo Sanguíneo_B: 0.7553786901176697
Grupo Sanguíneo_O: 0.028399819408446724
Fumante_Não: 2.050478434153999
Fumante_Sim: 1.94530811212543e-15
Nível de Atividade_Alto: 2.06799188359658
Nível de Atividade_Baixo: 0.0
Nível de Atividade_Moderado: 0.0


In [16]:
def performance_regressao(modelo, X_test, y_test):

    y_pred = modelo.predict(X_test)
    
    return mean_squared_error(y_test, y_pred, squared=False)

In [17]:
# Performance do modelo Lasso
performance_regressao(model_lasso, X_test, y_test)

9.062015729572144

In [18]:
# Treinar com o LassoCV
model_lasso_cv = LassoCV(alphas=[0.1, 0.5, 1], cv=5, random_state=51)
model_lasso_cv.fit(X_train, y_train)

LassoCV(alphas=[0.1, 0.5, 1], cv=5, random_state=51)

In [19]:
importancia_features(model_lasso_cv)

Importância das features:
Idade: 0.02398570771151734
Peso: 2.4765870268702703
Altura: 2.201760869534116
Grupo Sanguíneo_A: 0.12855801865958197
Grupo Sanguíneo_AB: 0.9304059600403655
Grupo Sanguíneo_B: 0.7553786901176697
Grupo Sanguíneo_O: 0.028399819408446724
Fumante_Não: 2.050478434153999
Fumante_Sim: 1.94530811212543e-15
Nível de Atividade_Alto: 2.06799188359658
Nível de Atividade_Baixo: 0.0
Nível de Atividade_Moderado: 0.0


In [20]:
performance_regressao(model_lasso_cv, X_test, y_test)

9.062015729572144

## Treinar modelo de regressão com Ridge (L2)

In [33]:
model_ridge = Ridge(alpha=0.1)
model_ridge.fit(X_train, y_train)

Ridge(alpha=0.1)

In [34]:
# importância das features - Lasso
importancia_features(model_ridge)

Importância das features:
Idade: 0.021562168148991458
Peso: 2.471277505688153
Altura: 2.198539914775216
Grupo Sanguíneo_A: 0.4425635910360846
Grupo Sanguíneo_AB: 1.3732758402349843
Grupo Sanguíneo_B: 1.3917552520269678
Grupo Sanguíneo_O: 0.4610430028161653
Fumante_Não: 1.2773055155977948
Fumante_Sim: 1.2773055155989128
Nível de Atividade_Alto: 1.7459863005674745
Nível de Atividade_Baixo: 0.7760548936045902
Nível de Atividade_Moderado: 0.9699314069799915


In [35]:
# Performance do modelo ridge
performance_regressao(model_ridge, X_test, y_test)

9.121486825982178

In [36]:
# Treinar com o LassoCV
model_lasso_cv = RidgeCV(alphas=[0.1, 0.5, 1], cv=5)
model_lasso_cv.fit(X_train, y_train)

RidgeCV(alphas=array([0.1, 0.5, 1. ]), cv=5)

In [37]:
importancia_features(model_lasso_cv)

Importância das features:
Idade: 0.02160242663673348
Peso: 2.4714395283499218
Altura: 2.19865426946907
Grupo Sanguíneo_A: 0.44196245948982715
Grupo Sanguíneo_AB: 1.3651312116461942
Grupo Sanguíneo_B: 1.3804905139799588
Grupo Sanguíneo_O: 0.4573217618213688
Fumante_Não: 1.2731125149106501
Fumante_Sim: 1.2731125149111377
Nível de Atividade_Alto: 1.7380726278216525
Nível de Atividade_Baixo: 0.7731441696098863
Nível de Atividade_Moderado: 0.9649284582135844


In [38]:
performance_regressao(model_lasso_cv, X_test, y_test)

9.12035069835535

In [ ]:
#